# Import an Xspec save file and recover the model parameters

We need to load the save file with pyxspec, then retrieve information from the xspec.AllData and xspec.AllModels objects.

The model parameters of the background components must be interpreted in tandem with their addmodel_* functions for consistency. Because we are building spectral and imaging products for these background components only, we will not consider any additional model components added by the user, who may need those to account for excess emission.

The user is supposed to not modify the loaded data files and existing model components (except their parameter values). For the spectral data files especially, their order must be preserved because this is used to match with their respective region files from bgdinfo.json. The loaded save file should then retain that same order for xspec.AllData.

In [1]:
import xspec

In [2]:
import json
import os
import astropy.io.fits as pf

## Load Xspec save file and bgdinfo.json

In [3]:
savefile = 'mymodel.xcm'
nustardir = '/apool/qw/astro/nustar'
specdir = nustardir + '/IC342_X1/90201039002/event_cl/spec'
bgddir = nustardir + '/IC342_X1/90201039002/event_cl/bgd'

In [4]:
os.chdir(bgddir)
xspec.Xset.restore(savefile)


***Warning: The file sent to Xset.restore(): mymodel.xcm
    is not detected to be a file generated from Xset.save().
    Xset.restore() usage is only intended for Xset.save() output.
    General XSPEC/Tcl scripts may not fully execute in PyXspec.




In [5]:
for i in range(xspec.AllData.nSpectra):
    spec = xspec.AllData(i + 1)
    print(spec.fileName)

bgd1A_sr_g30.pha
bgd1B_sr_g30.pha
bgd2A_sr_g30.pha
bgd2B_sr_g30.pha
bgd3A_sr_g30.pha
bgd3B_sr_g30.pha


In [6]:
xspec.AllModels.sources

{2: 'apbgd', 3: 'intbgd', 4: 'fcxb', 5: 'intn'}

Now read in the bgdinfo.json file for reference image and region files.

In [7]:
# from fitab.py
def check_bgdinfofile(bgdinfofile):
    """
    Check that the background info file has the required items.
    """
    if not os.path.exists(bgdinfofile):
        print('Error: file %s not found.' % bgdinfofile)
        return False

    bgdinfo = json.loads(open(bgdinfofile).read())

    problem = False
    for key in (
        'bgfiles', 'regfiles', 'refimgf', 'bgdapfiles', 'bgddetfiles'
    ):
        if key not in bgdinfo:
            problem = True
            print('%s not found in background info file.' % key)

    # Same number of items in bgfiles and regfiles:
    if len(bgdinfo['bgfiles']) != len(bgdinfo['regfiles']):
        problem = True
        print('bgfiles and regfiles must have the same number of entries.')

    # A and B keys in bgdapfiles and bgddetfiles
    if ('A' not in bgdinfo['bgdapfiles'] or
            'B' not in bgdinfo['bgdapfiles'] or
            'A' not in bgdinfo['bgddetfiles'] or
            'B' not in bgdinfo['bgddetfiles']):
        problem = True
        print('bgdapfiles and bgddetfiles must have A and B keys.')

    # Check files exist
    queue = []
    for _ in bgdinfo['bgfiles']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['regfiles']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['bgddetfiles']['A']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['bgddetfiles']['B']:
        if isinstance(_, str):
            queue.append(_)

    queue.append(bgdinfo['refimgf'])
    queue.append(bgdinfo['bgdapfiles']['A'])
    queue.append(bgdinfo['bgdapfiles']['B'])

    for _ in queue:
        if not os.path.exists(_):
            problem = True
            print('Error: file %s not found.' % _)

    if problem:
        return False
    else:
        return bgdinfo

In [8]:
bgdinfo = check_bgdinfofile('bgdinfo.json')

if bgdinfo is not False:
    # bgfiles and regfiles must have the same ordering
    bgfiles = bgdinfo['bgfiles']
    regfiles = bgdinfo['regfiles']
    refimgf = bgdinfo['refimgf']
    bgdapfiles = bgdinfo['bgdapfiles']
    bgddetfiles = bgdinfo['bgddetfiles']

    bgdapim = {}
    bgdapim['A'] = pf.open(bgdapfiles['A'])[0].data
    bgdapim['B'] = pf.open(bgdapfiles['B'])[0].data

    bgddetim = {}
    bgddetim['A'] = [
        pf.open(bgddetfiles['A'][0])[0].data,
        pf.open(bgddetfiles['A'][1])[0].data,
        pf.open(bgddetfiles['A'][2])[0].data,
        pf.open(bgddetfiles['A'][3])[0].data
    ]
    bgddetim['B'] = [
        pf.open(bgddetfiles['B'][0])[0].data,
        pf.open(bgddetfiles['B'][1])[0].data,
        pf.open(bgddetfiles['B'][2])[0].data,
        pf.open(bgddetfiles['B'][3])[0].data
    ]

In [9]:
bgdinfo

{'bgdapfiles': {'A': 'bgdapA.fits', 'B': 'bgdapB.fits'},
 'bgddetfiles': {'A': ['det0Aim.fits',
   'det1Aim.fits',
   'det2Aim.fits',
   'det3Aim.fits'],
  'B': ['det0Bim.fits', 'det1Bim.fits', 'det2Bim.fits', 'det3Bim.fits']},
 'bgfiles': ['bgd1A_sr_g30.pha',
  'bgd1B_sr_g30.pha',
  'bgd2A_sr_g30.pha',
  'bgd2B_sr_g30.pha',
  'bgd3A_sr_g30.pha',
  'bgd3B_sr_g30.pha'],
 'refimgf': 'bgdapA.fits',
 'regfiles': ['bgd1.reg',
  'bgd1.reg',
  'bgd2.reg',
  'bgd2.reg',
  'bgd3.reg',
  'bgd3.reg']}

We can check the ordering of the spectra files to make sure they match up.

In [10]:
for i in range(xspec.AllData.nSpectra):
    specfile = xspec.AllData(i + 1).fileName
    if specfile != bgdinfo['bgfiles'][i]:
        print('%s\tError: non-matching %s' % (specfile, bgdinfo['bgfiles'][i]))
    else:
        print('%s\t%s\tOK' % (specfile, bgdinfo['regfiles'][i]))

bgd1A_sr_g30.pha	bgd1.reg	OK
bgd1B_sr_g30.pha	bgd1.reg	OK
bgd2A_sr_g30.pha	bgd2.reg	OK
bgd2B_sr_g30.pha	bgd2.reg	OK
bgd3A_sr_g30.pha	bgd3.reg	OK
bgd3B_sr_g30.pha	bgd3.reg	OK


## apbgd component (unfocused aperture image background)

Because this background does not go through the optics, it is unfocused so we assume it has the same spectrum everywhere. The normalization rescaling between any two regions is the ratio of the total predicted flux in them, derived from the aspect-projected aperture background image (bgdapA.fits and bgdapB.fits).

Since all background spectra from each module are tied to one of them (as the reference spectrum) using the preset ratio in ratios.json, we can do this rescaling using any one spectrum from each module. For consistency, we will use the same reference spectra designation as when the model was generated.

## fcxb component (unresolved focused CXB background)

This background component is focused so it is expected to vary across the detector on the scale of the PSF, due to variance in the cosmic X-ray background. Each background region will have a different determination of the FCXB flux there.

By default, we will use an area-weighted average of the FCXB flux in all background regions and model it as constant everywhere. This is done by multiplying the combined aspect-projected detector masks by the average flux, and using this image to infer the total flux in the source region.

For imaging analysis, the user can generate a customized image of the FCXB flux and specify it to be used instead. This is also easily done after the fact by generating an image of just the FCXB component from the save file, and replacing this flux with user's custom image.

## intbgd component (instrumental background)

The instrumental background is modelled to be constant over each detector, and tied to one another by the weights in ratios.json.

We multiply each detector mask image by the flux on that detector, and create two aspect-projected detector background images, one for each module.

## intn component (neutron background)